In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from IPython.display import display
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import time
import glob
import sys
import os
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from hyperopt import hp, tpe

## Setup

In [ ]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
# import Dataset to play with it
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv('../input/test.csv')

In [ ]:
all_data = pd.concat([train_df, test_df], axis=0, sort=False)
scaler = StandardScaler()
for i in range(200):
    all_data['var_' + str(i)] = scaler.fit_transform(np.array(all_data['var_' + str(i)]).reshape(-1, 1))

In [ ]:
scale_train_df = all_data[all_data.target.notna()]
scale_test_df = all_data[all_data.target.isna()]

In [ ]:
%%time
idx = features = train_df.columns.values[2:202]

train_df['sum'] = scale_train_df[idx].sum(axis=1)  
train_df['min'] = scale_train_df[idx].min(axis=1)
train_df['max'] = scale_train_df[idx].max(axis=1)
train_df['mean'] = scale_train_df[idx].mean(axis=1)
train_df['std'] = scale_train_df[idx].std(axis=1)
train_df['skew'] = scale_train_df[idx].skew(axis=1)
train_df['kurt'] = scale_train_df[idx].kurtosis(axis=1)
train_df['med'] = scale_train_df[idx].median(axis=1)
    

In [ ]:
test_df['sum'] = scale_test_df[idx].sum(axis=1)  
test_df['min'] = scale_test_df[idx].min(axis=1)
test_df['max'] = scale_test_df[idx].max(axis=1)
test_df['mean'] = scale_test_df[idx].mean(axis=1)
test_df['std'] = scale_test_df[idx].std(axis=1)
test_df['skew'] = scale_test_df[idx].skew(axis=1)
test_df['kurt'] = scale_test_df[idx].kurtosis(axis=1)
test_df['med'] = scale_test_df[idx].median(axis=1)

In [ ]:
train_df.head()

In [ ]:
# train_df['scale_std'] = scale_train_df[idx].std(axis=1)
# test_df['scale_std'] = scale_test_df[idx].std(axis=1)

# train_df['scale_25'] = np.percentile(scale_train_df[idx], 25)
# test_df['scale_25'] = np.percentile(scale_test_df[idx], 25)

# train_df['scale_75'] = np.percentile(scale_train_df[idx], 75)
# test_df['scale_75'] = np.percentile(scale_test_df[idx], 75)

# train_df['scale_max'] = np.max(scale_train_df[idx])
# test_df['scale_max'] = np.max(scale_test_df[idx])

# train_df['scale_min'] = np.min(scale_train_df[idx])
# test_df['scale_min'] = np.min(scale_test_df[idx])

In [ ]:
# features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
# for feature in features:
#     train_df['r2_'+feature] = np.round(train_df[feature], 2)
#     test_df['r2_'+feature] = np.round(test_df[feature], 2)
#     train_df['r1_'+feature] = np.round(train_df[feature], 1)
#     test_df['r1_'+feature] = np.round(test_df[feature], 1)

In [ ]:
train_df.shape

In [ ]:
# train_df['var_108_cluster'] = train_df['var_108'].apply(lambda x: 0 if x < 14.2 else 1)
# train_df['var_141_cluster'] = train_df['var_141'].apply(lambda x: 0 if x < 8.9 else 1)
# train_df['var_181_cluster'] = train_df['var_181'].apply(lambda x: 0 if x < 10.8 else 1)
# train_df['var_83_cluster'] = train_df['var_83'].apply(lambda x: 0 if x < 2.5 else 1)
# train_df['var_123_cluster'] = train_df['var_123'].apply(lambda x: 0 if x < -7.2 else (1 if x < 6 else 2))

# test_df['var_108_cluster'] = test_df['var_108'].apply(lambda x: 0 if x < 14.2 else 1)
# test_df['var_141_cluster'] = test_df['var_141'].apply(lambda x: 0 if x < 8.9 else 1)
# test_df['var_181_cluster'] = test_df['var_181'].apply(lambda x: 0 if x < 10.8 else 1)
# test_df['var_83_cluster'] = test_df['var_83'].apply(lambda x: 0 if x < 2.5 else 1)
# test_df['var_123_cluster'] = test_df['var_123'].apply(lambda x: 0 if x < -7.2 else (1 if x < 6 else 2))

In [ ]:
# reverse_list = [0,1,2,3,4,5,6,7,8,11,15,16,18,19,
#             22,24,25,26,27,41,29,
#             32,35,37,40,48,49,47,
#             55,51,52,53,60,61,62,103,65,66,67,69,
#             70,71,74,78,79,
#             82,84,89,90,91,94,95,96,97,99,
#             105,106,110,111,112,118,119,125,128,
#             130,133,134,135,137,138,
#             140,144,145,147,151,155,157,159,
#             161,162,163,164,167,168,
#             170,171,173,175,176,179,
#             180,181,184,185,187,189,
#             190,191,195,196,199]

In [ ]:
# feature_cluster_config = {
#     'var_108': 3,
#     'var_41': 4,
#     'var_181': 4,
#     'var_83': 4,
#     'var_123': 4
# }
# n_cluster = 3

# for feat_val in reverse_list:
#     feature = 'var_' + str(feat_val)
#     clus_model = GaussianMixture(n_components=n_cluster, random_state=0, max_iter=10000, verbose=True)
#     clus_model.fit(train_df[[feature]])

#     train_proba_predict = clus_model.predict_proba(train_df[[feature]])
#     train_proba_df = pd.DataFrame(train_proba_predict)
#     train_proba_df.columns = [feature + '_clus_' + str(i) for i in range(n_cluster)]
#     train_df = pd.concat([train_df, train_proba_df], axis=1, sort=False)
    
#     test_proba_predict = clus_model.predict_proba(test_df[[feature]])
#     test_proba_df = pd.DataFrame(test_proba_predict)
#     test_proba_df.columns = [feature + '_clus_' + str(i) for i in range(n_cluster)]
#     test_df = pd.concat([test_df, test_proba_df], axis=1, sort=False)


In [ ]:
train_df.head()

## Model

In [ ]:
features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [ ]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold, StratifiedKFold

def objective(params):
    fold__ = 10
    param = {
        'num_leaves': int(params['num_leaves']),
        'min_data_in_leaf': int(params['min_data_in_leaf']), 
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'max_depth': int(params['max_depth']),
        'learning_rate': 0.015,
        "feature_fraction": params['feature_fraction'],
        'bagging_freq': 5,
        "bagging_fraction": params['bagging_fraction'] ,
        "bagging_seed": 11,
        'metric':'auc',
        'boosting' : 'gbdt',
        'boost_from_average':'false',
        "verbosity": -1,
        'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'seed':2019,
        'bagging_seed':2019,
        'drop_seed':2019,
        }
    folds = StratifiedKFold(n_splits=fold__, shuffle=False, random_state=44000)
    oof = np.zeros(len(train_df))
    # predictions = np.zeros(len(test_df))
    feature_importance_df = pd.DataFrame()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
        print("Fold {}".format(fold_))
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

        num_round = 30000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], \
                        early_stopping_rounds = 3000)
        
        oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
        
        del clf, trn_idx, val_idx
        gc.collect()
    return  -roc_auc_score(target, oof)

In [ ]:
from hyperopt.fmin import fmin

space = {
    'num_leaves': hp.quniform('num_leaves', 2, 12, 2),
    'max_depth': hp.quniform('max_depth', -1, 10, 1),
    'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 15, 1),
    'feature_fraction': hp.quniform('feature_fraction', 0.02, 0.1, 0.005),
    'bagging_fraction': hp.quniform('bagging_fraction', 0.1, 0.5, 0.05),
    'min_data_in_leaf': hp.quniform('min_data_in_leaf', 50, 100, 5)

}
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=5)

In [ ]:
print("Hyperopt estimated optimum {}".format(best))

In [ ]:
# from bayes_opt import BayesianOptimization
# from sklearn.model_selection import KFold, StratifiedKFold
# fold__ = 10
# def LGB_CV(
#           max_depth,
#           num_leaves,
#           min_data_in_leaf,
#           feature_fraction,
#           bagging_fraction,
#           min_sum_hessian_in_leaf
#          ):
    
#     folds = StratifiedKFold(n_splits=fold__, shuffle=False, random_state=44000)
#     oof = np.zeros(len(train_df))
#     # predictions = np.zeros(len(test_df))
#     feature_importance_df = pd.DataFrame()

#     for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
#         print("Fold {}".format(fold_))
#         param = {
#             'num_leaves': int(num_leaves),
#             'min_data_in_leaf': int(min_data_in_leaf), 
#             'min_sum_hessian_in_leaf': min_sum_hessian_in_leaf,
#             'max_depth': int(max_depth),
#             'learning_rate': 0.01,
#             "feature_fraction": feature_fraction,
#             'bagging_freq': 5,
#             "bagging_fraction": bagging_fraction ,
#             "bagging_seed": 11,
#             'metric':'auc',
#             'boosting' : 'gbdt',
#             'boost_from_average':'false',
#             "verbosity": -1,
#             'num_threads': 8,
#             'tree_learner': 'serial',
#             'objective': 'binary', 
#             'seed':int(2**fold__),
#             'bagging_seed':int(2**fold__),
#             'drop_seed':int(2**fold__)
#         }
#         trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
#         val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

#         num_round = 30000
#         clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, \
#                         early_stopping_rounds = 3000)
        
#         oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
        
#         del clf, trn_idx, val_idx
#         gc.collect()
        
#     return roc_auc_score(target, oof)

In [ ]:
# LGB_BO = BayesianOptimization(LGB_CV, {
#     'max_depth': (2, 10),
#     'num_leaves': (5, 130),
#     'min_data_in_leaf': (50, 150),
#     'feature_fraction': (0.2, 0.5),
#     'bagging_fraction': (0.2, 0.5),
#     'min_sum_hessian_in_leaf':(5, 15)
# })

# print('-'*126)

# with warnings.catch_warnings():
#     warnings.filterwarnings('ignore')
#     LGB_BO.maximize(init_points=2, n_iter=10, acq='ei', xi=0.0)
    
# LGB_BO.max

In [ ]:
# print(LGB_BO.max)

In [ ]:
# param = {
#     'bagging_freq': 5,
#     'bagging_fraction': 0.4,
#     'boost_from_average':'false',
#     'boost': 'gbdt',
#     'feature_fraction': 0.05,
#     'learning_rate': 0.01,
#     'max_depth': 7,  
#     'metric':'auc',
#     'min_data_in_leaf': 80,
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_leaves': 21,
#     'num_threads': 8,
#     'tree_learner': 'serial',
#     'objective': 'binary', 
#     'verbosity': 1
# }

In [ ]:
# folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
# oof = np.zeros(len(train_df))
# predictions = np.zeros(len(test_df))
# feature_importance_df = pd.DataFrame()

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
#     print("Fold {}".format(fold_))
#     trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
#     val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])

#     num_round = 1000000
#     clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, \
#                     early_stopping_rounds = 3000)
#     oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["Feature"] = features
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_ + 1
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
#     predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

# print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [ ]:
# cols = (feature_importance_df[["Feature", "importance"]]
#         .groupby("Feature")
#         .mean()
#         .sort_values(by="importance", ascending=False)[:150].index)
# best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

# plt.figure(figsize=(14,28))
# sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
# plt.title('Features importance (averaged/folds)')
# plt.tight_layout()
# plt.savefig('FI.png')

In [ ]:
# sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
# sub_df["target"] = predictions
# sub_df.to_csv("submission.csv", index=False)